In [ ]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Ramping with I
ISIs = [0.4] * 4
ISI_pts = np.cumsum(ISIs)

# Constants
I0 = 1.5
beta = 3.0
alpha = 1.0

# Initialize
IS = I0
IP = I0
y0 = 0.7

# For storage
yS_lst = []
IS_lst = []
IP_lst = []
productions = []

# Synchronization
for i in range(len(ISIs)):
    print('iteration #', i, 'current ISI = ', ISI_pts[i])
    # Period adjustment
    yS = IS * ISIs[i]
    IS_new = IS - beta * (yS - y0)
    print('yS = ', yS, 'IS_new = ', IS_new)
    yS_lst.append(yS)
    IS_lst.append(IS_new)
    
    # Make production
    tp = y0 / IP
    productions.append(tp)
    print('tp =  ', tp)
    
    # Phase adjustments
    # Calculate y_out of SAM
    if i == 0:
        y_out = IS_new * (np.sum(productions)) # Assume ISI_pts[i-1] = 0 here
    else:
        if ISI_pts[i] < np.sum(productions):
            y_out = IS_new * (np.sum(productions) - ISI_pts[i-1])
        else:
            y_out = IS_new * (np.sum(productions) - ISI_pts[i-1])
    
    print('y_out = ', y_out)
    
    IP = IS_new - alpha * (y0 - y_out)
    IS = IS_new
    IP_lst.append(IP)
    print('IP = ', IP)
    
    
    
# Continuation
n_continuation = 5
yS_inf = 1.0
for i in range(n_continuation):
    # Period adjustment: IS unchanged since no stimulus is observed
    yout = yS_inf
    
    # Make production
    tp = y0 / IP
    productions.append(tp)
    print('tp =  ', tp)
    
    # Phase adjustments
    IP = IS - alpha * (y0 - yout)
    IP_lst.append(IP)
    print('IP = ', IP)
    

    








In [ ]:
# Plot the result
plt.plot(np.cumsum(productions), np.ones(len(productions)), '.')
plt.plot(ISI_pts, np.ones(len(ISI_pts)) * 1.0, 'r.')

## Discrete updating algorithm

### Continuous time definitions
$y_{\textrm{ref}}$ : Reference level

$y_{\textrm{SAM}}$ : Output of SAM

$y_{\textrm{MPM}}$ : Output of of MPM

$I_{\textrm{SAM}}$ : Input to SAM

$I_{\textrm{MPM}}$ : Input to MPM


### Translation into discrete time
$i$ will index *time-points* and *intervals*

$t_p^i$ : time of $i$th production

$t_s^i$ : time of $i$th stimulus

$\textrm{ISI}^i$ = $t_s^{i+1} - t_s^i$

$\textrm{IPI}^i$ = $t_p^{i+1} - t_p^i$

$y_{\textrm{SAM}}^i=I_{\textrm{SAM}}^i*\textrm{ISI}^i$

$I_{\textrm{SAM}}^{i+1}=I_{\textrm{SAM}}^i+\beta(y_{\textrm{ref}}-y_{\textrm{SAM}}^i)$

$I_{\textrm{MPM}}^{i+1} = I_{\textrm{SAM}}^{i+1} + \alpha(y_{\textrm{out}}^i-y_{\textrm{MPM}}^i)$

$\textrm{IPI}^i=y_{\textrm{ref}}/I_{\textrm{MPM}}$

$t_p^{i+1}=t_p^i+\textrm{IPI}^i$

By definition $y_{\textrm{MPM}}^i=y_{\textrm{ref}}$ (output of $y_{\textrm{MPM}}$ at ith production)

The value of $y_{\textrm{out}}^i$ represents the output of the SAM at the time of the $i$th production. During synchronization that will be:

if $t_s^{i}<t_p^i$

$y_{\textrm{out}}^i = I_{\textrm{SAM}}^i*(t_p^{i}-t_s^{i})$

else

$y_{\textrm{out}}^i = I_{\textrm{SAM}}^i*(t_p^i-t_s^{i-1})$


During continuation:

$y_{\textrm{out}}^i = y_{\textrm{SAM}}^\infty$

with $y_{\textrm{SAM}}^\infty$ representing the terminal output of the SAM module.

In [ ]:
# Definitions
def I_s_update(I_s_in,beta,y_ref,y_s):
    I_s_out = I_s_in + beta*(y_ref - y_s)
    
    return I_s_out

def I_p_update(I_s,alpha,y_p,y_out):
    I_p_out = I_s + alpha*(y_out-y_p)
    
    return I_p_out

def discreteAlgorithm(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=0.8, y_s_infty=1.5):

    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    IPI = np.zeros((len(t_s)))
    t_p = np.zeros((len(t_s)))
    I_s = np.zeros((len(t_s)))
    I_p = np.zeros((len(t_s)))
    y_s = np.zeros((len(t_s)))
    I_p[0] = I_s_init
    I_s[0] = I_s_init

    for i in range(len(ISI)):

        IPI[i] = y_ref/I_p[i]
        if i > 0:
            t_p[i+1] = t_p[i] + IPI[i]
        else:
            t_p[i+1] = IPI[i]
        
        if np.isnan(ISI[i]):
            y_s[i] = y_s_infty
            I_s[i+1] = I_s[i]
        else:
            y_s[i] = I_s[i]*ISI[i]
            I_s[i+1] = I_s_update(I_s[i],beta,y_ref,y_s[i])

        
        if np.isnan(ISI[i]):
            y_out = y_s_infty
        else:
            if t_s[i] < t_p[i]:
                y_out = I_s[i]*(t_p[i]-t_s[i])
            else:
                if i == 0:
                    y_out = y_ref
                else:
                    y_out = I_s[i]*(t_p[i]-t_s[i-1])
                #print(i,y_out)


        I_p[i+1] = I_p_update(I_s[i+1],alpha,y_ref,y_out)
        #print(t_s[i],t_p[i],IPI[i])
    
    return t_s, t_p, IPI, I_s, I_p, y_s

def discreteAlgorithm_modified(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=0.8):

    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    IPI = np.zeros((len(t_s)))
    t_p = np.zeros((len(t_s)))
    I_s = np.zeros((len(t_s)))
    I_p = np.zeros((len(t_s)))
    y_s = np.zeros((len(t_s)))
    I_p[0] = I_s_init
    I_s[0] = I_s_init

    for i in range(len(ISI)):

        IPI[i] = y_ref/I_p[i]
        if i > 0:
            t_p[i+1] = t_p[i] + IPI[i]
        else:
            t_p[i+1] = IPI[i]

        y_s[i] = I_s[i]*ISI[i]
        #print('i = ', i,'y_s = ', y_s[i], 'I_s = ', I_s[i])

        I_s[i+1] = I_s_update(I_s[i],beta,y_ref,y_s[i])

        if t_s[i] < t_p[i]:
            y_out = I_s[i]*(t_p[i]-t_s[i-1])
        else:
            if i == 0:
                y_out = y_ref
            else:
                y_out = I_s[i]*(t_p[i]-t_s[i-1])
            #print(i,y_out)


        #print('yout = ', y_out)
        I_p[i+1] = I_p_update(I_s[i+1],alpha,y_ref,y_out)
        #print(t_s[i],t_p[i],IPI[i])
    
    return t_s, t_p, IPI, I_s, I_p, y_s

## Synchronization to a constant ISI

### Algorithm with $t_p[i]-t_s[i]$ and $t_p[i]-t_s[i-1]$

In [ ]:
# Simple synchronization
ISI_short = [0.5] * 10
ISI_long = [0.7] * 10

t_s1, t_p1, IPI1, _, _, _ = discreteAlgorithm(ISI_short, beta=2.0, alpha=0.1, y_ref=1, I_s_init=1/0.6)
t_s2, t_p2, IPI2, _, _, _ = discreteAlgorithm(ISI_long, beta=2.0, alpha=0.1, y_ref=1, I_s_init=1/0.6)



In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(121)
plt.plot(t_s1, np.ones(len(t_s1)), '.')
plt.plot(t_p1, np.ones(len(t_s1)),'r.')
plt.title('ISI = 0.5')

plt.subplot(122)
plt.plot(t_s2, np.ones(len(t_s2)), '.')
plt.plot(t_p2, np.ones(len(t_s2)),'r.')
plt.title('ISI = 0.7')
plt.legend(['Stimulus', 'Production'])

In [ ]:
plt.plot(t_p1-t_s1)
plt.plot(t_p2 - t_s2)
plt.legend(['ISI = 0.5', 'ISI = 0.7'])

### Algorithm with $t_p[i]-t_s[i-1]$ only

In [ ]:
# Simple synchronization
ISI_short = [0.5] * 10
ISI_long = [0.7] * 10

t_s1, t_p1, IPI1, _, _, _ = discreteAlgorithm_modified(ISI_short, beta=2.0, alpha=0.1, y_ref=1, I_s_init=1/0.6)
t_s2, t_p2, IPI2, _, _, _ = discreteAlgorithm_modified(ISI_long, beta=2.0, alpha=0.1, y_ref=1, I_s_init=1/0.6)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(121)
plt.plot(t_s1, np.ones(len(t_s1)), '.')
plt.plot(t_p1, np.ones(len(t_s1)),'r.')
plt.title('ISI = 0.5')

plt.subplot(122)
plt.plot(t_s2, np.ones(len(t_s2)), '.')
plt.plot(t_p2, np.ones(len(t_s2)),'r.')
plt.title('ISI = 0.7')
plt.legend(['Stimulus', 'Production'])

In [ ]:
plt.plot(t_p1-t_s1)
plt.plot(t_p2 - t_s2)
plt.legend(['ISI = 0.5', 'ISI = 0.7'])

## Discrete updating algorithm (Including Nhat's revision)

### Continuous time definitions
$y_{\textrm{ref}}$ : Reference level

$y_{\textrm{SAM}}$ : Output of SAM

$y_{\textrm{MPM}}$ : Output of of MPM

$I_{\textrm{SAM}}$ : Input to SAM

$I_{\textrm{MPM}}$ : Input to MPM


### Translation into discrete time
$i$ will index *time-points* and *intervals*

$t_p^i$ : time of $i$th production

$t_s^i$ : time of $i$th stimulus

$\textrm{ISI}^i$ = $t_s^{i+1} - t_s^i$

$\textrm{IPI}^i$ = $t_p^{i+1} - t_p^i$

$y_{\textrm{SAM}}^i=I_{\textrm{SAM}}^i*\textrm{ISI}^i$

$I_{\textrm{SAM}}^{i+1}=I_{\textrm{SAM}}^i+\beta(y_{\textrm{ref}}-y_{\textrm{SAM}}^i)$

$I_{\textrm{MPM}}^{i+1} = I_{\textrm{SAM}}^{i+1} + \alpha(y_{\textrm{out}}^i-y_{\textrm{MPM}}^i)$

$\textrm{IPI}^i=y_{\textrm{ref}}/I_{\textrm{MPM}}$

$t_p^{i+1}=t_p^i+\textrm{IPI}^i$

By definition $y_{\textrm{MPM}}^i=y_{\textrm{ref}}$ (output of $y_{\textrm{MPM}}$ at ith production)

The value of $y_{\textrm{out}}^i$ represents the output of the SAM at the time of the $i$th production. During synchronization that will be:

$y_{\textrm{out}}^i = I_{\textrm{SAM}}^i*(t_p^i-t_s^{i-1})$

During continuation:

$y_{\textrm{out}}^i = y_{\textrm{SAM}}^\infty$

with $y_{\textrm{SAM}}^\infty$ representing the terminal output of the SAM module.

In [ ]:
# Definitions
def I_s_update(I_s_in,beta,y_ref,y_s):
    I_s_out = I_s_in + beta*(y_ref - y_s)
    
    return I_s_out

def I_p_update(I_s,alpha,y_p,y_out):
    I_p_out = I_s + alpha*(y_out-y_p)
    
    return I_p_out

def discreteAlgorithm(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=0.8, y_s_infty=1.5):

    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    IPI = np.zeros((len(t_s)))
    t_p = np.zeros((len(t_s)))
    I_s = np.zeros((len(t_s)))
    I_p = np.zeros((len(t_s)))
    y_s = np.zeros((len(t_s)))
    I_p[0] = I_s_init
    I_s[0] = I_s_init

    for i in range(len(ISI)):

        IPI[i] = y_ref/I_p[i]
        if i > 0:
            t_p[i+1] = t_p[i] + IPI[i]
        else:
            t_p[i+1] = IPI[i]
        
        if np.isnan(ISI[i]):
            y_s[i] = y_s_infty
            I_s[i+1] = I_s[i]
        else:
            y_s[i] = I_s[i]*ISI[i]
            I_s[i+1] = I_s_update(I_s[i],beta,y_ref,y_s[i])

        
        if np.isnan(ISI[i]):
            y_out = y_s_infty
        else:
            if i == 0:
                y_out = y_ref
            else:
                y_out = I_s[i]*(t_p[i]-t_s[i-1])
            #print(i,y_out)

        # Standard code for algorithm, implementing integration of differences over time
        I_p[i+1] = I_p_update(I_s[i+1],alpha,y_ref,y_out)
        
        # Code that does not integrate differences over time
        #if np.isnan(t_s[i]):
        #    I_p[i+1] = I_p_update(I_s[i+1],alpha,t_p[i],t_p[i]+0.4)
        #else:       
        #    I_p[i+1] = I_p_update(I_s[i+1],alpha,t_p[i],t_s[i])
        
        #print(t_s[i],t_p[i],IPI[i])
    
    return t_s, t_p, IPI, I_s, I_p, y_s

## Synchronization to a constant ISI

In [ ]:
# Simple synchronization
ISI = [0.4] * 500
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.2, alpha=0.1, y_ref=1, I_s_init=1/0.35)

#for i in range(len(t_s)):
#    print(t_s[i],t_p[i],IPI[i])

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

## Step Change

In [ ]:
# Step change
ISI = np.concatenate((0.80*np.ones(100),1.00*np.ones(30)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.5, alpha=0.1, y_ref=1, I_s_init=1/0.8)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),1.00*np.ones(9)))
betas = [0.5,0.7,0.9]
alpha = 0.3
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((14,len(betas)))
for betai in range(len(betas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=betas[betai], alpha=alpha, y_ref=1, I_s_init=1/0.8)
    TS[:,betai] = t_s
    TP[:,betai] = t_p
    IPIs[:,betai] = IPI
    

In [ ]:
print(ISI,IPIs[0:-1,0])
plt.plot(ISI)
plt.plot(IPIs[0:-1,:])

In [ ]:
plt.plot(TP-TS)

## Phase shift

In [ ]:
# Phase shift
ISI = np.concatenate((0.80*np.ones(50),0.9*np.ones(1),0.80*np.ones(50)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.7, alpha=0.1, y_ref=1, I_s_init=1/0.8)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),0.9*np.ones(1),0.80*np.ones(8)))
beta = 0.7
alphas = [0.1,0.2,0.3]
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((14,len(betas)))
for alphai in range(len(alphas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alphas[alphai], y_ref=1, I_s_init=1/0.8)
    TS[:,alphai] = t_s
    TP[:,alphai] = t_p
    IPIs[:,alphai] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPIs)

In [ ]:
plt.plot(TP-TS)

## Stimulus jitter

In [ ]:
# Stimulus jitter
ISI = np.concatenate((0.80*np.ones(50),0.9*np.ones(1), 0.7*np.ones(1), 0.80*np.ones(49)))
del t_p
del t_s
del IPI
del I_s
del I_p
del y_s
t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.7, alpha=0.1, y_ref=1, I_s_init=1/0.8)

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
ISI = np.concatenate((0.80*np.ones(4),0.9*np.ones(1), 0.7*np.ones(1), 0.80*np.ones(7)))
beta = 0.7
alphas = [0.1,0.2,0.3]
TS = np.zeros((14,len(betas)))
TP = np.zeros((14,len(betas)))
IPIs = np.zeros((14,len(betas)))
for alphai in range(len(alphas)):
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alphas[alphai], y_ref=1, I_s_init=1/0.8)
    TS[:,alphai] = t_s
    TP[:,alphai] = t_p
    IPIs[:,alphai] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPIs)

In [ ]:
plt.plot(TP-TS)

## 1-2-Go, 1-2-3-Go

In [ ]:
def simulate_123Go(nstages=3,beta=0.75,alpha=0.0,ISI0=0.8,y_ref=1,ISImin=0.600,ISImax=1.000,number=5):
    # 1-2-Go, 1-2-3-Go
    ISIs = np.linspace(ISImin,ISImax,num=number)
    ISI_list = np.nan*np.ones((len(ISIs),nstages))
    IPI_list = np.nan*np.ones((len(ISIs),nstages+1))
    t_s_list = []
    t_p_list = []
    for i in range(len(ISIs)):
        ISI_list[i,:] = ISIs[i]*np.ones(nstages)
        ISI = ISI_list[i,:]
        t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alpha, y_ref=y_ref, I_s_init=1/ISI0)

        t_s_list.append(t_s)
        t_p_list.append(t_p)
        IPI_list[i,:] = IPI
    
    return IPI_list, ISIs

IPI_list, ISIs = simulate_123Go()

In [ ]:
print(IPI_list.shape)
tempISIs = np.matlib.repmat(ISIs,4,1)
sqErrs = np.square(IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)
print(sqBIAS)

In [ ]:
plt.subplot(121)
plt.plot(sqBIAS[1:3])

plt.subplot(122)
plt.plot(np.transpose(tempISIs),IPI_list)

## Synchronization/continuation

In [ ]:
# Synchronization/continuation
ISIs = np.linspace(0.550,0.817,num=5)
print(ISIs)
ISI_list = np.nan*np.ones((len(ISIs),12))
IPI_list = np.nan*np.ones((len(ISIs),13))
t_s_list = []
t_p_list = []
for i in range(len(ISIs)):
    ISI_list[i,:] = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(8)))
    del t_p
    del t_s
    del IPI
    del I_s
    del I_p
    del y_s
    ISI = ISI_list[i,:]
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=0.75, alpha=0.1, y_ref=1, I_s_init=1/0.6835, y_s_infty=1.5)
    
    t_s_list.append(t_s)
    t_p_list.append(t_p)
    IPI_list[i,:] = IPI

In [ ]:
plt.plot(ISI)
plt.plot(IPI)

In [ ]:
plt.plot(t_p-t_s)

In [ ]:
tempISIs = np.matlib.repmat(ISIs,13,1)
sqErrs = np.square(IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)
print(sqBIAS)

In [ ]:
plt.subplot(121)
plt.plot(sqBIAS[1:12])

plt.subplot(122)
print(tempISIs.shape)
plt.plot(np.transpose(tempISIs[(0,4,10),:]),IPI_list[:,(0,4,10)])

### Fit to behavior

#### Synchronization/continuation

In [ ]:
import scipy.io
import glob

def find_bias_arr(filename, tmax):
    '''filename: SynCon_ITIs file name
    tmax: max number of taps to consider
    
    Returns: the bias array, tmax x number of intervals in prior'''
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()
    splits = data['allDur_splits']
    IPI_mean = data['allDur_mean'][0]
    
    Bias = np.zeros((tmax, 5))

    for i in range(len(IPI_mean)):
        IPI_single = IPI_mean[i].flatten()[:tmax]
        bias_single = IPI_single - durs[i]
        Bias[:,i] = bias_single
    
    return Bias

def plot_subject_ts_tp(filename):
    bias_arr = find_bias_arr(filename, 20)
    std_arr = find_std_arr(filename, 20)
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()

    # Plot ts-tp dependence (subject)
    bias_first = bias_arr[0,:]
    std_first = std_arr[0,:]
    bias_third = bias_arr[2,:]
    std_third = std_arr[2,:]
    bias_fifth = bias_arr[4,:]
    std_fifth = std_arr[4,:]

    plt.errorbar(durs, durs + bias_third, std_third)
    plt.errorbar(durs, durs + bias_fifth, std_fifth)
    plt.plot(durs, durs, '--')

def find_std_arr(filename, tmax):
    '''filename: SynCon_ITIs file name
    tmax: max number of taps to consider
    
    Returns: the bias array, tmax x number of intervals in prior'''
    data = scipy.io.loadmat(filename)
    durs = data['durs'].flatten()
    splits = data['allDur_splits']
    IPI_std = data['allDur_std'][0]
    
    STD = np.zeros((tmax, 5))

    for i in range(len(IPI_std)):
        std_single = IPI_std[i].flatten()[:tmax]
        STD[:,i] = std_single
    
    return STD
    
def minimizer(filename,ISIs,nsamples,betaRange,alphaRange,ISI0Range):
    bias_arr = find_bias_arr(filename, 20)
    
    betas = np.random.uniform(low=betaRange[0], high=betaRange[1], size=nsamples)
    alphas = np.random.uniform(low=alphaRange[0], high=alphaRange[1], size=nsamples)
    ISI0s = np.random.uniform(low=ISI0Range[0], high=ISI0Range[1], size=nsamples)
    mse = np.nan*np.ones((nsamples,1))
    for j in range(nsamples):
        ISI_list = np.nan*np.ones((len(ISIs),21))
        ISI_list2 = np.nan*np.ones((len(ISIs),21))
        IPI_list = np.nan*np.ones((len(ISIs),22))
        t_s_list = []
        t_p_list = []
        for i in range(len(ISIs)):
            ISI_list[i,:] = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(17)))
            ISI_list2[i,:] = ISIs[i]*np.ones(21)
            ISI = ISI_list[i,:]
            t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=betas[j], alpha=alphas[j], y_ref=1, I_s_init=1/ISI0s[j], y_s_infty=1.5)

            t_s_list.append(t_s)
            t_p_list.append(t_p)
            IPI_list[i,:] = IPI
            
        biasSim = np.sqrt( np.nanmean((IPI_list[:,1:-1]-ISI_list2[:,1:])**2,axis=0) )
        biasObs = np.sqrt( np.nanmean(bias_arr**2,axis=1) )
        mse[j] = np.nanmean((biasObs - biasSim)**2)
           
    id = np.argmin(mse)
    print(id)
    beta = betas[id]
    alpha = alphas[id]
    ISI0 = ISI0s[id]
    return beta, alpha, ISI0

In [ ]:
ISIs = np.linspace(0.550,0.817,num=5)
dirlst = glob.glob('/Users/swe/Dropbox (MIT)/CircuitModel/Data/*_2_*ITIs.mat')
filename = dirlst[4] #'/Users/swe/Dropbox (MIT)/CircuitModel/Data/FK_2_20170721_SynCon_ITIs.mat'
print(filename)
beta, alpha, ISI0 = minimizer(filename, ISIs,nsamples=1000, betaRange=[0,0.8], alphaRange=[0,0.8], ISI0Range=[0.5,0.9])


In [ ]:
print(beta,alpha,ISI0)

In [ ]:
ISI_list = np.nan*np.ones((len(ISIs),12))
IPI_list = np.nan*np.ones((len(ISIs),13))
t_s_list = []
t_p_list = []
for i in range(len(ISIs)):
    ISI_list[i,:] = np.concatenate((ISIs[i]*np.ones(4),np.nan*np.ones(8)))
    del t_p
    del t_s
    del IPI
    del I_s
    del I_p
    del y_s
    ISI = ISI_list[i,:]
    t_s, t_p, IPI, I_s, I_p, y_s = discreteAlgorithm(ISI, beta=beta, alpha=alpha, y_ref=1, I_s_init=1/ISI0, y_s_infty=1.5)
    
    t_s_list.append(t_s)
    t_p_list.append(t_p)
    IPI_list[i,:] = IPI


bias_arr = find_bias_arr(filename, 20)
biasObs = np.sqrt(np.nanmean(bias_arr**2,axis=1))
tempISIs = np.matlib.repmat(ISIs,13,1)
sqErrs = np.square(IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)

plt.subplot(121)
plt.plot(np.sqrt(sqBIAS[1:12]))
plt.plot(biasObs[0:11])

plt.subplot(122)
plt.plot(np.transpose(tempISIs[(3,5),:]),IPI_list[:,(3,5)])
plot_subject_ts_tp(filename)

In [ ]:
BETAS = np.zeros((len(dirlst),1))
ALPHAS = np.zeros((len(dirlst),1))
ISIinits = np.zeros((len(dirlst),1))
for filei in range(len(dirlst)):
    filename = dirlst[filei] #'/Users/swe/Dropbox (MIT)/CircuitModel/Data/FK_2_20170721_SynCon_ITIs.mat'
    print(filename)
    beta, alpha, ISI0 = minimizer(filename, ISIs,nsamples=1000, betaRange=[0,0.8], alphaRange=[0,0.8], ISI0Range=[0.5,0.9])
    
    BETAS[filei] = beta
    ALPHAS[filei] = alpha
    ISIinits[filei] = ISI0

In [ ]:
print(BETAS,ALPHAS,ISIinits)

#### 1-2-Go/1-2-3-Go

In [ ]:
def get_time_lst(nstages, beta, alpha, ISI0):
    '''Simulate and return the array of times of threshold crossing'''
    IPI_list, ISIs = simulate_123Go(nstages=nstages,beta=beta,alpha=alpha,ISI0=ISI0)
    times = 1000*IPI_list[:,-2:-1]

    return times

def do_beta_alpha_ISI0_fitting(subject_file, nsamples, betalow, betahigh, alphalow, alphahigh, ISI0low, ISI0high,
                 print_every=10):
    '''Inputs:
    - subject_file: .mat file with the behavioral results of the subject
    - nsamples: number of random (beta,alpha,ISI0) combinations to sample
    - betalow, betahigh, alphalow, alphahigh, ISI0low, ISI0high: ranges of beta, alpha, and ISI0 for sampling
    
    Outputs:
    beta, alpha, ISI0: the optimal combination to minimize the mse between the mean simulated times
    and the mean behavioral times of the subject'''
    #print('sigmaval = ', sigmaval)
    betalst = np.random.uniform(low=betalow, high=betahigh, size=nsamples)
    alphalst = np.random.uniform(low=alphalow, high=alphahigh, size=nsamples)
    ISI0lst = np.random.uniform(low=ISI0low, high=ISI0high, size=nsamples)

    # Load data for subject SWE(RSG/RSSG, Seth provided)
    swedata = scipy.io.loadmat(subject_file)
    B = swedata['B']
    ts_vec = swedata['ts_vec']
    ta = swedata['ta']
    tss = swedata['tss']
    mtp_in = swedata['mtp_in']
    stdtp_in = swedata['stdtp_in']

    target_time1 = mtp_in[:,0]
    target_time2 = mtp_in[:,1]

    b = np.mean(B[:,0])

    durations = tss
    errorlst = []

    for k in range(nsamples):
        ISI0val = ISI0lst[k]
        betaval = betalst[k]
        alphaval = alphalst[k]

        prod_time_lst, prod_var_lst, prod_time_lst2, prod_var_lst2 = [], [], [], []

        #1,2,Go simulation
        prod_time_lst = get_time_lst(nstages=2, beta=betaval, alpha=alphaval, ISI0=ISI0val)

        #1,2,3,Go simulation
        prod_time_lst2 = get_time_lst(nstages=3, beta=betaval, alpha=alphaval, ISI0=ISI0val)

        # Continue if there is a nan   
        error1 = (np.transpose(prod_time_lst) - target_time1) ** 2
        error2 = (np.transpose(prod_time_lst2) - target_time2) ** 2
        
        if np.sum(np.isnan(error1)) + np.sum(np.isnan(error2)) > 0:
            error = np.inf
            print('nan encountered')
        else:
        # Error function to minimize
            error = np.sum((np.transpose(prod_time_lst) - target_time1) ** 2) +\
                    np.sum((np.transpose(prod_time_lst2) - target_time2) ** 2)
        errorlst.append(error)

        #if k % print_every == 0: # Print every 10 lines
        #    print('    alpha = %f, beta = %f, ISI0 = %f, error = %f' %(alphaval, betaval, ISI0val, error))
    
    # Find the optimal
    plt.scatter(betalst,alphalst,c=np.sqrt(errorlst))
    idx = np.argmin(errorlst)
    return betalst[idx], alphalst[idx], ISI0lst[idx]

In [ ]:
dirlst = glob.glob('/Users/swe/Dropbox (MIT)/CircuitModel/Data/*_EKF_*.mat')
filename = dirlst[-2]
print(filename)
beta, alpha, ISI0 = do_beta_alpha_ISI0_fitting(filename, nsamples=10000, betalow=0.0, betahigh=0.9, alphalow=0, alphahigh=0.01, ISI0low=0.500, ISI0high=1.100)


In [ ]:
print(filename)
swedata = scipy.io.loadmat(filename)
mtp_in = swedata['mtp_in']


print(alpha,beta,ISI0)
IPI_list, ISIs = simulate_123Go(nstages=3,beta=beta,alpha=alpha,ISI0=ISI0)

tempISIs = np.matlib.repmat(1000*ISIs,4,1)
sqErrs = np.square(1000*IPI_list - np.transpose(tempISIs))
sqBIAS = sqErrs.sum(axis=0)/len(ISIs)

plt.subplot(121)
plt.plot(sqBIAS[1:3])

plt.subplot(122)
plt.plot(np.transpose(tempISIs[:2,:]),1000*IPI_list[:,1:3])
plt.plot(np.transpose(tempISIs[:2,:]),mtp_in)

In [ ]:
BETAS = np.zeros((len(dirlst),1))
ALPHAS = np.zeros((len(dirlst),1))
ISIinits = np.zeros((len(dirlst),1))
IPImodel = np.nan*np.ones((5,2,len(dirlst)))
IPIsubjects = np.nan*np.ones((5,2,len(dirlst)))
IPIsubjects_std = np.nan*np.ones((5,2,len(dirlst)))
for filei in range(len(dirlst)):
    filename = dirlst[filei] #'/Users/swe/Dropbox (MIT)/CircuitModel/Data/FK_2_20170721_SynCon_ITIs.mat'
    print(filename)
    swedata = scipy.io.loadmat(filename)
    mtp_in = swedata['mtp_in']
    stdtp_in = swedata['stdtp_in']
    beta, alpha, ISI0 = do_beta_alpha_ISI0_fitting(filename, nsamples=1000, betalow=0.1, betahigh=0.9, alphalow=0.01, alphahigh=0.9, ISI0low=0.600, ISI0high=1.000)
    
    BETAS[filei] = beta
    ALPHAS[filei] = alpha
    ISIinits[filei] = ISI0
    
    IPI_list, ISIs = simulate_123Go(nstages=3,beta=beta,alpha=alpha,ISI0=ISI0)
    
    IPImodel[:,:,filei] = IPI_list[:,1:3]
    IPIsubjects[:,:,filei] = mtp_in
    IPIsubjects_std[:,:,filei] = stdtp_in

In [ ]:
print(IPImodel[:,:,0],BETAS[0])

In [ ]:
scipy.io.savemat('/Users/swe/Dropbox (MIT)/CircuitModel/ModelResults/discreteAlgorithm_123Go.mat', 
                 {'IPIsubjects': IPIsubjects, 'IPIsubjects_std': IPIsubjects_std,
                 'IPImodel': IPImodel, 'BETAS': BETAS,
                 'ALPHAS': ALPHAS, 'INIinits': ISIinits})

# Continuous time algorithm

In [ ]:
def integratorModule(y,I,dt):
    y = y + I*dt
    
    return y

def continuousTimeAlgorithm(ISI,beta,alpha,I0,dt):
    
    totalT = sum(ISI)
    t_s = np.zeros(len(ISI)+1)
    t_s[1:len(ISI)+1] = np.cumsum(ISI)
    t_p = np.zeros(len(ISI)+1)
    t_p_ind = 0
    t_s_ind = 0
    tSteps = int(totalT/dt)
    
    y_s = np.zeros(tSteps+1)
    y_p = np.zeros(tSteps+1)
    I = I0*np.ones(tSteps+1)
    I[0] = I0
    I_p = I
    
    for ti in range(tSteps):
        if t_s[t_s_ind] < ti*dt:
            simulatorError = 1 - y_s[ti]
            I[ti] = I[ti-1] + beta*simulatorError  # Update input into integrator input
            y_s[ti+1] = 0         # Reset simulator on stimulus input
            t_s_ind = t_s_ind+1
        else:
            I[ti] = I[ti-1]
            y_s[ti+1] = integratorModule(y_s[ti],I[ti],dt)
            
        deltaI = alpha*(y_s[ti]-y_p[ti])
        I_p[ti] = I[ti]+deltaI
        
        if y_p[ti] > 1:
            t_p[t_p_ind] = ti*dt
            t_p_ind = t_p_ind+1
            y_p[ti+1] = 0
        else:
            y_p[ti+1] = integratorModule(y_p[ti],I_p[ti],dt)
        
        
    return y_s, y_p, I, I_p, t_p, t_s

In [ ]:
ISI = [0.4] * 50
y_s, y_p, I, I_p, t_p, t_s = continuousTimeAlgorithm(ISI,beta=0.5,alpha=0,I0=1/0.600,dt=0.01)

In [ ]:
IPI = np.diff(t_p)
plt.plot(y_p)